In [1]:
#Get dependencies 

import pickle
from string import Template
import geocoder

from string import Template
import json
from urllib.request import urlopen
import urllib

import re
import pickle



# To be set in a json

Cameroun = ["Yaoundé"]
#, "Douala", "Bamenda", "Bafoussam", "Ngaoundéré", "Nkongsamba", "Belel", "Garoua", "Maroua"]

liste_pays = {"Cameroun": Cameroun}



country = 1
menu =""
dict_country = ""
dict_menu= ""
dicte = ""
dict_gps = ""
coo = {}
bbox = {}


for i in liste_pays:

    city = 1
    Input_p = {"id": str(country), "i": i }
    
    menu += Template('{id: "$id", name:"$i" },').safe_substitute(Input_p)
    dict_country += Template('"$id":"$i",').safe_substitute(Input_p)
    dict_menu_t= ''
    
    for j in liste_pays[i]:
        g = geocoder.google(j)
        
        t =g.latlng
        k= str(country)+"_"+str(city)
        
        Input_c = {"k":k, "i": j, "gps": t}
        
        
        
        dict_menu_t += Template('{id: "$k", name:"$i" },').safe_substitute(Input_c)
        dicte +=  Template('"$k":"$i",').safe_substitute(Input_c)
        dict_gps +=  Template('"$k":$gps,').safe_substitute(Input_c)

        coo[j] =  t
        bbox[j] = [t[0]-0.5, t[1]-0.5, t[0]+0.5, t[1]+0.5]
        
        city +=1 
        
        
    dict_menu += '"'+i+'":['+dict_menu_t+'],'
    country +=1
    
    selected_country = '"'+i+'"'
    selected_city = '"' + j + '"'
    selected_gps = t


dict_menu = "{"+dict_menu[:-1]+"}"
dict_gps = "{"+dict_gps[:-1]+"}"
dicte = "{"+dicte[:-1]+"}"
menu= "["+menu+"]"
dict_country = "{"+dict_country[:-1]+'}'



In [5]:
http = 'http://overpass-api.de/api/interpreter?'


dict_query = {'road' : '''[out:json]; way["highway"="trunk"]$bbox;
      out geom body qt; relation["highway"="hospital"]$bbox; out geom body qt; way["highway"="primary"]$bbox;
      out geom body qt; relation["highway"="primary"]$bbox; out geom body qt;
      '''  }

for i in bbox:
    temp = str(bbox[i])
    bb = "("+temp[1:-1]+")"
    Input = {'bbox': bb}
    for z in dict_query:
        query = Template(dict_query[z]).safe_substitute(Input)
        
        print(query)
        query_string = urllib.parse.urlencode({'data': query})
        query_string = query_string.encode('utf-8')
        print(type(query_string))

        try:
            response = urlopen(url=http, data=query_string).read()
        except :
            print("Oups")

        Input_save = {'query' : z, 'nam' : i}
        name = Template('$nam-$query-center.json').safe_substitute(Input_save)
        with open(name, "w") as f: f.write(response.decode('utf-8'))

            
#json still need to be processed with bash + osmtogeojson 
# IN BASH: for file in *.json; do osmtogeojson  "$file" >  "${file%.json}.geojson" ; done

[out:json]; way["highway"="trunk"](3.3480325, 11.0020752, 4.3480325, 12.0020752);
      out geom body qt; relation["highway"="hospital"](3.3480325, 11.0020752, 4.3480325, 12.0020752); out geom body qt; way["highway"="primary"](3.3480325, 11.0020752, 4.3480325, 12.0020752);
      out geom body qt; relation["highway"="primary"](3.3480325, 11.0020752, 4.3480325, 12.0020752); out geom body qt;
      
<class 'bytes'>
